In [ ]:
# OpenMV MobileNetV2 int8 Inference Script
# Requires mobilenetv2_224_int8.tflite in the filesystem

import sensor, image, time, tf, pyb

# === SETTINGS ===
MODEL_PATH = "mobilenetv2_224_int8.tflite"
LABELS = ["NO_THREAT", "THREAT"]
IMG_SIZE = 224

# === CAMERA INIT ===
sensor.reset()
sensor.set_pixformat(sensor.RGB565)  # OpenMV uses RGB565 internally
sensor.set_framesize(sensor.QVGA)    # 320x240
sensor.skip_frames(time=2000)
sensor.set_auto_gain(False)          # IMPORTANT: disable auto gain
sensor.set_auto_whitebal(False)      # IMPORTANT: disable white balance
clock = time.clock()

# === LOAD TFLITE MODEL ===
net = tf.load(MODEL_PATH, load_to_fb=True)  # load_to_fb helps RAM usage

while True:
    clock.tick()
    img = sensor.snapshot()

    # Resize to match MobileNetV2 input size
    resized = img.resize(IMG_SIZE, IMG_SIZE)

    # Run inference
    out = net.classify(resized)[0].output()

    # Get best prediction
    max_index = max(range(len(out)), key=lambda i: out[i])
    label = LABELS[max_index]
    confidence = out[max_index] / 255.0  # int8 outputs are 0–255

    # Print results to IDE terminal
    print("Prediction:", label, " Confidence:", confidence)

    # Draw on frame
    img.draw_string(0, 0, f"{label} ({confidence:.2f})", color=(255, 0, 0), scale=2)

    # Optional: blink LED for THREAT detection
    if label == "THREAT" and confidence > 0.8:
        pyb.LED(1).on()
    else:
        pyb.LED(1).off()

    print("FPS:", clock.fps())
